In [1]:
from langchain.document_loaders import WebBaseLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.retrievers.multi_query import MultiQueryRetriever
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
import tiktoken
import bs4
from langchain.prompts.chat import ChatPromptTemplate

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
OPENAI_MODEL = "gpt-3.5-turbo"
OPENAI_EMBEDDING_MODEL = "text-embedding-3-small"

In [3]:
loader = WebBaseLoader(
    web_paths=[
        "https://g1.globo.com/rs/rio-grande-do-sul/noticia/2024/05/15/levantamento-enchente-porto-alegre-bairros-e-moradores.ghtml",
        "https://www.cnnbrasil.com.br/nacional/enchente-em-porto-alegre-veja-imagens-da-cidade-apos-queda-no-nivel-da-agua/",
        "https://www.bbc.com/portuguese/articles/cw00d51k5rlo",
        "https://gauchazh.clicrbs.com.br/ultimas-noticias/tag/alagamentos/",
        "https://www.poder360.com.br/infraestrutura/especialistas-listam-medidas-para-evitar-enchentes-em-porto-alegre/"
    ]
)
documents = loader.load()

In [4]:
type(documents[0])

langchain_core.documents.base.Document

In [5]:
encoding = tiktoken.encoding_for_model(OPENAI_MODEL)
tiktoken_len = lambda text: len(encoding.encode(text))

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=tiktoken_len,
    add_start_index=True
)

In [7]:
chunks = text_splitter.split_documents(documents)

In [8]:
vector_db_floods_rs = Chroma.from_documents(documents=chunks, embedding=OpenAIEmbeddings(model=OPENAI_EMBEDDING_MODEL), persist_directory="./vector_db_floods_rs")

In [9]:
llm_retriever = ChatOpenAI(temperature=0, model=OPENAI_MODEL)
retriever_from_llm = MultiQueryRetriever.from_llm(
    retriever=vector_db_floods_rs.as_retriever(top_k=10), llm=llm_retriever
)

In [10]:
type(retriever_from_llm)

langchain.retrievers.multi_query.MultiQueryRetriever

In [11]:
RAG_PROMPT = """
CONTEXTO:
{context}

PERGUNTA DO USUÁRIO:
{question}

O contexto descrito acima foi recuperado através de uma RAG. Caso o contexto esteja vazio ou seja inútil, responda com "Eu não sei". Caso contrário, utilize o contexto 
obtido para responder a pergunta do usuário. 
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [12]:
llm_answer = ChatOpenAI(model="gpt-4o")

In [13]:
rag_chain = (
    {"context": retriever_from_llm, "question": RunnablePassthrough()}
    | rag_prompt
    | llm_answer
    | StrOutputParser()
)

In [14]:
rag_chain.invoke("Como evitar enchentes em Porto Alegre?")

'Para evitar enchentes em Porto Alegre, os especialistas sugerem uma série de medidas emergenciais e de longo prazo:\n\n### Medidas Emergenciais:\n1. **Vedação das Comportas**:\n   - Utilizar mergulhadores e sacos permeáveis para vedar as comportas do muro e da Avenida Castelo Branco.\n\n2. **Instalação de Ensecadeiras**:\n   - Instalar ensecadeiras em todas as 23 casas de bombas para que possam voltar a funcionar.\n   - Reenergizar os locais usando redes paralelas de cabos isolados ou geradores.\n   - Caso as casas de bombas não possam ser operadas, utilizar bombas volantes de grande vazão para drenar o centro histórico e a região norte.\n\n3. **Vedação das Tampas dos Condutos Forçados**:\n   - Vedação das tampas violadas dos Condutos Forçados Polônia e Álvaro Chaves.\n\n### Medidas para Quando o Nível da Água Estiver Baixo:\n1. **Manutenção de Comportas**:\n   - Consertar e substituir comportas danificadas.\n\n2. **Regularização das Casas de Bombas**:\n   - Contratar a regularização 

In [15]:
rag_chain.invoke("Qual o verdadeiro nome do Homem-Aranha?")

'Eu não sei.'